
# Regression for Boundary Value Problems in Solid Mechanics

Authors: Vivek Chavan (Github: Vivek9Chavan), Chi Shing Li (Github: charles4444), Ahmet Küpeli

Date: 15.01.2020

This Script was created for the course Computational Intelligence in Engineering hold by Arndt Koeppe and Marion Mundt, supervised by Prof. Bernd Markert @ Institue of General Mechanics RWTH Aachen University 

# Test tensorflow gpu check:

In [ ]:
import tensorflow as tf 
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name())) 
else: print("Please install GPU version of TF")


# Import

In [1]:
%matplotlib notebook
import os
import glob
import numpy as np
import tensorflow as tf
import matplotlib

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPool2D, TimeDistributed, LSTM
import matplotlib

from matplotlib import pyplot as plt
from helpers import DataReader, plot_sample

#Importing all functions from metrics, just in case:
from metrics import mean_absolute_percentage_error, max_relative_error, symmetric_mean_absolute_percentage_error

# Data

In [3]:
# PLEASE CHECK THE PATH
tfrecord_dir = os.path.join(r'C:_path', 'data', 'tfrecord')
tfrecord_files = glob.glob(os.path.join(tfrecord_dir, '*'))
#Data
data_format = 'NSXYF'     # Axes in data: Samples (N), Sequence (S), X-Axis (X), Y-Axis (Y), Features (F)
data_shape = [None, 100, 9, 9, 21]
idx = {'Xi_boundary': [0, 1],
       'Ui': [8, 9],
       'Fi': [16, 17],
       'boundary': [4],
       'Xi': [6, 7],
       'Ui_boundary': [2, 3],
       'body': [5],
       'Si': [10, 11, 12, 13]}
input_features = idx['Ui'] + idx['Si']  
output_features = idx['Fi']


# Hyper parameters - Batch sizes & Learning rates
learning_rate=[0.001]
batch_size_loop=[16]
#learning_rate=[0.0001, 0.001,0.01,0.1]
#batch_size_loop=[8,16,32,64,128]

#data split
tf.random.set_seed(123)
tfrecord_files=tf.random.shuffle(tfrecord_files)

'Data Split is approx. 70-15-15'
train_data, valid_data, test_data = tfrecord_files[:45000], tfrecord_files[45000:55000], tfrecord_files[55000:]

# Calculate Mean and SD for hardcoding

##  Do not run it unless necessary. The computation takes long time! The Datasets will also need to be plotted before running this.

Following is an example of how the Mean and the SD can be calculated for a given feature. Here it will be calculated for the input feature only!



def mean_fun(x):
    x_abs = tf.abs(x)
    full_mask = tf.cast(tf.sign(x_abs), tf.bool)
    sequence_mask = tf.cast(tf.reduce_any(full_mask, axis=(2, 3, 4)), tf.float32)
    mean, var = tf.nn.weighted_moments(x, frequency_weights=tf.cast(full_mask, tf.float32), axes=(0, 1, 2, 3))
    std = tf.sqrt(var)
    x1 = mean
    return x1

def std_fun(x):
    x_abs = tf.abs(x)
    full_mask = tf.cast(tf.sign(x_abs), tf.bool)
    mean, var = tf.nn.weighted_moments(x, frequency_weights=tf.cast(full_mask, tf.float32), axes=(0, 1, 2, 3))
    std = tf.sqrt(var)
    x = std
    return x

def preprocess_output0(y):
    mean = tf.convert_to_tensor([0.0, 0.0])
    std = tf.convert_to_tensor([1.0, 1.0])
    y = (y - mean) / std
    return y

def mean_and_dummy(*batch):
    return (mean_fun(batch[0]), preprocess_output0(batch[1]))

def std_and_dummy(*batch):
    return (std_fun(batch[0]), preprocess_output0(batch[1]))

#not sure if the data is getting reshuffled again!
tf.random.set_seed(123)

mean_set = train_dataset.map(mean_and_dummy)

std_set = train_dataset.map(std_and_dummy)

summe1=0
sumsd1=0
tf0=0

#VARY THIS TO CHANGE THE NUMBER OF BATCHES OVER WHICH IT IS CALCULATED
no_batches=1

# Final Mean Calculation

for batch in mean_set.take(no_batches):
    summe=sum(batch[0])
    #print(batch[0])
    summe1=summe+summe1
    #tfsum= tf.math.reduce_mean(batch[0], keepdims=False)
    #print('Tensorflow mean is:')
    #print(tfsum)

    
for batch in std_set.take(no_batches):
    sumsd=sum(batch[0])
    #print(batch[0])
    sumsd1=sumsd+sumsd1

#print(summe1)
mean_value=summe1/(no_batches*2)
print('The mean is:')
print(mean_value)

# Final Std calculation
#print(sumsd1)
std_value=sumsd1/(no_batches*2)
print('The SD is:')
print(std_value)

 # Mean and SD:

After running the calculations the final valies for stress (Si) are:

##  Ui
Mean= 0

SD = 0.001

##  Si
Mean= 0.011196767

SD = 4.3331237

##  Fi
Mean= 0

SD = 0.37

## These must now be used for hardcoding in the preprocessing below!

# Preprocessing

In [5]:
#Prepocessing: THIS IS CORRECT PREPROCESSING!!
def preprocess_input(x):
    # mean = tf.reduce_mean(x, axis=(0, 1, 2, 3), keepdims=True)
    mean = tf.convert_to_tensor([0, 0, 0.011196767, 0.011196767, 0.011196767, 0.011196767], np.float32)
    std = tf.convert_to_tensor([0.001, 0.001, 4.3331237, 4.3331237, 4.3331237, 4.3331237], np.float32)
    x = (x - mean) / std
    return x

def preprocess_output(y):
    mean = tf.convert_to_tensor([0, 0], np.float32)
    std = tf.convert_to_tensor([0.37, 0.37], np.float32)
    y = (y - mean) / std
    return y

def preprocess_fun(*batch):
    return (preprocess_input(batch[0]), preprocess_output(batch[1]))


# Reference Architecture - AlexNet

In [ ]:
#Reference Architecture - AlexNet
history_list=[]
test_loss_list=[]
kernel_size=[2,2]
epochs=80

# for loops - Batch sizes
for i in range(0,len(batch_size_loop)):
    data_reader = DataReader(data_shape, data_format,  batch_size=batch_size_loop[i], src_type='file',
                         slice_tensors=False, buffer_size=1000,
                         input_features=input_features, output_features=output_features,
                         sample_dtype=tf.float64, cast_dtype=tf.float32)
    
    train_dataset = data_reader.generate_batch_dataset(train_data, batch_size_loop[i])
    valid_dataset = data_reader.generate_batch_dataset(valid_data, batch_size_loop[i])
    test_dataset = data_reader.generate_test_dataset(test_data, batch_size_loop[i])
    processed_train_dataset = train_dataset.map(preprocess_fun)
    processed_valid_dataset = valid_dataset.map(preprocess_fun)
    processed_test_dataset = test_dataset.map(preprocess_fun)
    
    # for loops - Learning rates
    for j in range(0,len(learning_rate)):
        tf.keras.backend.clear_session()
        def create_AlexNET_func(kernel_size,batch_size_loop):
            # Input layer:
            input_shape = list(data_shape[1:-1]) + [len(input_features)]
            inputs = tf.keras.Input(input_shape,batch_size=batch_size_loop)
            x = inputs
            
            # Hidden layers:
            #1st Convolutional Layer
            x=TimeDistributed(Conv2D(filters=4, kernel_size=kernel_size, padding='same', activation='relu'))(x)
            x=TimeDistributed(MaxPool2D(pool_size=(2,2), padding='valid'))(x)
    
            #2nd Conv. Layer
            x = TimeDistributed(Conv2D(filters=10, kernel_size=kernel_size, padding='same', activation='relu'))(x)
            x=TimeDistributed(MaxPool2D(pool_size=(2,2), padding='valid'))(x)
    
            #3rd Conv. Layer
            x= TimeDistributed(Conv2D(filters=15,  kernel_size=kernel_size, padding='same', activation='relu'))(x)

            #Flattening
            x=TimeDistributed(Flatten())(x)
    

            #1st Dense Layer
            x=TimeDistributed(Dense(240, activation='relu'))(x)
    
            #2nd Dense Layer
            x=TimeDistributed(Dense(162, activation='linear'))(x)
            x = tf.keras.layers.TimeDistributed(tf.keras.layers.Reshape((9,9,len(output_features))))(x)
            #x = tf.keras.layers.Masking(mask_value=0.0)(x)
            
            # Output layer
            return tf.keras.models.Model(inputs=inputs, outputs=x)
        
        #Training
        validation_steps= len(valid_data) / (batch_size_loop[i]*epochs)
        steps_per_epoch= len(train_data) / (batch_size_loop[i]* epochs)

        model=create_AlexNET_func(kernel_size,batch_size_loop[i])

        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate[j])
        model.compile(loss='mae', optimizer=opt)
        history=model.fit(processed_train_dataset,validation_data=processed_valid_dataset,validation_steps=validation_steps,shuffle=True ,steps_per_epoch=steps_per_epoch, epochs=epochs)
        history_list.append(history)
        
        #Testing
        test_loss=model.evaluate(processed_test_dataset)
        test_loss_list.append("Batch= " +str(batch_size_loop[i])+ "; Learning= " + str(learning_rate[j])+ "; Test Loss= " + str(test_loss))
        model.save("AlexNET_Model_15_Jan" + str(i) + ".h5")

        
        #Vizualisation - traing loss vs validation loss
        #plt.title('AlexNet Losses')
        #plt.ylabel('Loss')
        #plt.xlabel('Epoch')        
        #plt.plot(history.history['loss'])
        #plt.plot(history.history['val_loss'])
        #plt.legend(['Training','Validation'],loc='upper left')
        #plt.grid(axis='both',which='major')
        #plt.savefig('Fig_alexFinal_B' + str(batch_size_loop[i]) + "l" + str(learning_rate[j]) + ".png")        
        #plt.clf()


print(test_loss_list)


# Own Architecture - LSTM RNN

In [ ]:
# Own Architecture - LSTM RNN
history_list=[]
test_loss_list=[]
epochs=80

# for loops - Batch sizes
for i in range(0,len(batch_size_loop)):
    data_reader = DataReader(data_shape, data_format,  batch_size=batch_size_loop[i], src_type='file',
                         slice_tensors=False, buffer_size=1000,
                         input_features=input_features, output_features=output_features,
                         sample_dtype=tf.float64, cast_dtype=tf.float32)
    
    train_dataset = data_reader.generate_batch_dataset(train_data, batch_size_loop[i])
    valid_dataset = data_reader.generate_batch_dataset(valid_data, batch_size_loop[i])
    test_dataset = data_reader.generate_test_dataset(test_data, batch_size_loop[i])
    processed_train_dataset = train_dataset.map(preprocess_fun)
    processed_valid_dataset = valid_dataset.map(preprocess_fun)
    processed_test_dataset = test_dataset.map(preprocess_fun)
    
    # for loops - Learning rates
    for j in range(0,len(learning_rate)):
        tf.keras.backend.clear_session()
        def create_RecNET(batch_size):
            input_shape = list(data_shape[1:-1]) + [len(input_features)]
            
            # Input layer
            inputs = tf.keras.Input(input_shape,batch_size=batch_size)
            x = inputs
            
            # Hidden layers:
            # Flatten layer - flatten 5d tensor into 3d tensor which is the required input for lstm layer
            x = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(x)  
            #x = SequenceMasking(mask_value=0.)(x)
    
            # 1st LSTM layer
            x = tf.keras.layers.LSTM(324, activation='tanh', return_sequences=True, dropout=0.1)(x)
   
            # 2nd LSTM layer
            x = tf.keras.layers.LSTM(216, activation='tanh', return_sequences=True, dropout=0.1)(x)
        
            # 3rd LSTM layer
            #x = tf.keras.layers.LSTM(216, activation='tanh', return_sequences=True, dropout=0.1)(x)
   

            #x=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(334, activation='relu'))(x)
            #x= Dropout(0.2)(x)

            # Dense layer - linear activation - make sure all output between +ve and -ve
            x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(162, activation='linear'))(x)
            # Reshape layer - reshape 3d tensor back to 5d tensor
            x = tf.keras.layers.TimeDistributed(tf.keras.layers.Reshape((9,9,len(output_features))))(x)
            #x = tf.keras.layers.Masking(mask_value=0.)(x)
            
            # Output layers
            return tf.keras.models.Model(inputs=inputs, outputs=x)
        
        # Training
        validation_steps= len(valid_data) / (batch_size_loop[i]*epochs)
        steps_per_epoch= len(train_data) / (batch_size_loop[i]* epochs)

        model=create_RecNET(batch_size_loop[i])
        model.summary()

        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate[j])
        model.compile(loss='mae', optimizer=opt)
        history=model.fit(processed_train_dataset,validation_data=processed_valid_dataset,validation_steps=validation_steps,shuffle=True ,steps_per_epoch=steps_per_epoch, epochs=epochs)
        history_list.append(history)
        
        #Testing
        test_loss=model.evaluate(processed_test_dataset)
        test_loss_list.append("Batch= " +str(batch_size_loop[i])+ "; Learning= " + str(learning_rate[j])+ "; Test Loss= " + str(test_loss))
        #model.save("RecNET_Model_15_Jan" + str(i) + ".h5")
        
        #Vizualisation - traing loss vs validation loss
        #plt.title('RecNet Losses')
        #plt.ylabel('Loss')
        #plt.xlabel('Epoch')        
        #plt.plot(history.history['loss'])
        #plt.plot(history.history['val_loss'])
        #plt.legend(['Training','Validation'],loc='upper left')
        #plt.grid(axis='both',which='major')
        #plt.savefig('Fig_RecFinal_B' + str(batch_size_loop[i]) + "l" + str(learning_rate[j]) + ".png")        
        #plt.clf()

# Create a Numpy array for Test Dataset

In [ ]:
#These functions will be used for generating a target set

def dummy_fun_ip(x):
    mean = tf.convert_to_tensor([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    std = tf.convert_to_tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
    x=(x-mean)/std
    return x

def dummy_fun_op(x):
    mean = tf.convert_to_tensor([0.0, 0.0])
    std = tf.convert_to_tensor([1.0, 1.0])
    x=(x-mean)/std
    return x    

def preprocess_tf(*batch):
    return (dummy_fun_ip(batch[0]), dummy_fun_op(batch[1]))

processed_test_dataset2 = test_dataset.map(preprocess_tf)

nn_target = [batch[1] for batch in processed_test_dataset2]
target = zip(*nn_target)
target = [np.concatenate(o, axis=0) for o in target]

# Get Target set maskm for calculating SMAPPE:
target_mask=tf.abs(target)
target_mask= tf.cast(tf.sign(target), tf.bool)

target2=tf.boolean_mask(target, target_mask)
    
np.shape(target)

# Data postprocessing

In [ ]:
def postprocess_output(y_):
    mean = tf.convert_to_tensor([0, 0], np.float32)
    std = tf.convert_to_tensor([0.37, 0.37], np.float32)
    y_ = (y_ * std) + mean
    return y_

#Creating a similar postprocessing function
def postprocess_fun(*batch):
    return (postprocess_output(batch[0]))

# SMAPE Calculation

In [ ]:
nn_output = [model.predict(batch) for batch in processed_test_dataset]
output = [postprocess_fun(nno) for nno in nn_output]
output = zip(*output)
output = [np.concatenate(o, axis=0) for o in output]
np.shape(output)

#Symmetric Mean Absolute Percentage Error (SMAPE)
output2=tf.boolean_mask(output, target_mask)

smape = symmetric_mean_absolute_percentage_error(
target2, output2, reduction_axes=None, norm_mode='range')

print('The SMAPE is: ')
print(smape)

# Visualization - Results

In [ ]:
# Prediction vs Target
for iter in range(20):
    
    sample = target[3]
    tmp2 = sample[0+(500*iter):500+(500*iter), 0, 0, 0]
    print(tmp2.shape)
    fig = plt.figure(figsize=(7,3))
    plt.plot(tmp2, 'r')

    sample3 = output[3]
    tmp3 = sample3[0+(500*iter):500+(500*iter), 0, 0, 0]
    plt.plot(tmp3, 'g')
    plt.legend(['Target','Prediction'],loc='lower right')
    plt.savefig('Alex_Seq'+ str(iter)+'.png') 

In [ ]:
# Heat map of Target

for iter in range(20):
    idx_op={'Fi': [0,1]}
    sample = target[3]
    tmp = sample[2000+iter,..., idx_op['Fi'][0]]
    plt.figure(figsize=(4, 4))
    plt.imshow(tmp)

    #plt.yticks(range(-3,3))
    plt.colorbar()
    plt.draw()
    plt.savefig('Alex_Tar'+ str(iter)+'.png') 

In [ ]:
# Heatmap of prediction 

for iter in range(20):
    sample = output[3]
    tmp = sample[2000+iter,..., idx_op['Fi'][0]]
    plt.figure(figsize=(4, 4))
    plt.imshow(tmp)
    plt.colorbar()
    plt.draw()
    plt.savefig('Alex_Out'+ str(iter)+'.png')